In [1]:
import numpy as np
execfile('midi_to_state.py')
daft_state = midiToNoteStateMatrix('daft_punk-one_more_time.mid')


In [47]:
def encode_midi_states(state_mat):
    chord_to_idx = {}
    idx_to_chord = {}
    for sixteenth_note in state_mat:
        notes_only = [note[0] for note in sixteenth_note]
        note_idcs_set = tuple([i for i, x in enumerate(notes_only) if x == 1])
        if len(note_idcs_set) == 0: note_idcs_set = (None)
        if not note_idcs_set in chord_to_idx.keys():
            new_idx = len(chord_to_idx) + 1
            chord_to_idx[note_idcs_set] = new_idx
            idx_to_chord[new_idx] = note_idcs_set
        else:
            note_set_idx = chord_to_idx[note_idcs_set]
    return [chord_to_idx, idx_to_chord]

results = encode_midi_states(daft_state)
print (len(results[0]))
print (len(results[1]))

381
381


In [48]:
import random
randnum = random.sample(range(len(results[0])),1)[0]
print randnum
assert(results[0][results[1][randnum]] == randnum)

315


In [51]:
# now the set of chords is a library of possible classes
# return a num_notes x len_chord_set
def encode_song(midi_states, chord_to_idx):
    song_encoded = np.zeros((len(midi_states)+1, len(chord_to_idx)+1))
    for index in range(len(midi_states)):
        notes_only = [note[0] for note in midi_states[index]]
        note_idcs_set = tuple([i for i, x in enumerate(notes_only) if x == 1])
        if len(note_idcs_set) == 0: note_idcs_set = (None)
        library_idx = chord_to_idx[note_idcs_set]
        song_encoded.itemset((index,library_idx), 1)
    return song_encoded

encoded_song = encode_song(daft_state, results[0])
    

In [54]:
randnum = random.sample(range(len(encoded_song)),1)[0]
print randnum
assert(sum(encoded_song[randnum]) == 1)

543


In [55]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

maxlen = 25
chars = set(results[0].keys())
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

Using Theano backend.
00001	

/Users/aimeebarciauskas/.theano/compiledir_Darwin-14.5.0-x86_64-i386-64bit-i386-2.7.11-64/cutils_ext/mod.cpp:740:11: error: use of undeclared identifier 'import_array'
          import_array();
          ^
1 error generated.




00002	        #include <Python.h>
00003	        #include "numpy/arrayobject.h"
00004	        #include "theano_mod_helper.h"
00005	
00006	        extern "C"{
00007	        static PyObject *
00008	        run_cthunk(PyObject *self, PyObject *args)
00009	        {
00010	          PyObject *py_cthunk = NULL;
00011	          if(!PyArg_ParseTuple(args,"O",&py_cthunk))
00012	            return NULL;
00013	
00014	          if (!PyCObject_Check(py_cthunk)) {
00015	            PyErr_SetString(PyExc_ValueError,
00016	                           "Argument to run_cthunk must be a PyCObject.");
00017	            return NULL;
00018	          }
00019	          void * ptr_addr = PyCObject_AsVoidPtr(py_cthunk);
00020	          int (*fn)(void*) = (int (*)(void*))(ptr_addr);
00021	          void* it = PyCObject_GetDesc(py_cthunk);
00022	          int failure = fn(it);
00023	
00024	          return Py_BuildValue("i", failure);
00025	         }
00026	        #if NPY_API_VERSION >= 0x00000008
00027	        t

Exception: Compilation failed (return status=1): /Users/aimeebarciauskas/.theano/compiledir_Darwin-14.5.0-x86_64-i386-64bit-i386-2.7.11-64/cutils_ext/mod.cpp:740:11: error: use of undeclared identifier 'import_array'.           import_array();.           ^. 1 error generated.. 